<a href="https://colab.research.google.com/github/robertoricci/Curso-Python-para-Mercado-Financeiro-Trading-com-Dados/blob/main/03_An%C3%A1lise_T%C3%A9cnica_com_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Instalando Bibliotecas


In [1]:
!pip install ta
#!pip install investpy
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ta: filename=ta-0.10.2-py3-none-any.whl size=29106 sha256=c38c9adfa076a35d7f35b6cfa4d94c2ed8b87281dea31759a73a2c837824d521
  Stored in directory: /root/.cache/pip/wheels/31/31/f1/f2ff471bbc5b84a4b973698ceecdd453ae043971791adc3431
Successfully built ta
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 642 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:
!pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import ta
#import investpy
import pandas as pd
import plotly.graph_objects as go
import yfinance as yf

##2. Capturando Dados

In [ ]:
#df = investpy.get_stock_historical_data(stock='BRML3', country='Brazil', from_date='01/01/2019', to_date='30/03/2021')

In [4]:
df = yf.download('BRML3.SA', start='2019-01-01', end='2021-03-30')

[*********************100%***********************]  1 of 1 completed


In [5]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,13.07,13.51,12.96,13.46,12.450921,14193800
2019-01-03,13.46,13.56,13.22,13.55,12.534173,9265200
2019-01-04,13.42,13.59,13.29,13.29,12.293665,11161000
2019-01-07,13.16,13.30,12.98,13.21,12.219663,4569500
2019-01-08,13.26,13.30,13.10,13.23,12.238163,4509200
...,...,...,...,...,...,...
2021-03-23,9.52,9.84,9.39,9.68,9.625318,14677500
2021-03-24,9.77,10.10,9.68,9.74,9.684978,27053200
2021-03-25,9.71,9.95,9.48,9.82,9.764525,23272200


##3. Indicadores de Momentum

####3.1. RSI - Relative Strenght Index

100 - [100+ 1[media dos ganhos/media das perdas]

Media dos ganhos dos ultimos dos 14 períodos = 0.7%
Media das perdas dos ultimos dos 14 períodos = -0.9%

100 - [100+ 1[(0.7/14)/(-0.9/14)]]

In [6]:
from ta.momentum import RSIIndicator
from plotly.subplots import make_subplots

In [7]:
#Inicializando indicador RSI

rsi = RSIIndicator(close=df['Close'],window=30)
df['RSI'] = rsi.rsi()

In [8]:
#Gráfico Preços/RSI

fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(name='RSI',x=df.index, y=df['RSI']), row=2, col=1)
fig.add_trace(go.Candlestick(name='BRML3',x=df.index, close=df['Close'], open=df['Open'],high=df['High'], low=df['Low']),row=1,col=1)
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

###3.2. Percentage Price Oscillator (PPO)
####verificar se o preço esta sobre comprado ou sobre vendido

In [9]:
from ta.momentum import PercentagePriceOscillator

In [10]:
PPO = PercentagePriceOscillator(df['Close'])
df['PPO'] = PPO.ppo()
df['PPO_hist'] = PPO.ppo_hist()
df['PPO_line'] = PPO.ppo_signal()

In [11]:
#grafico segundario
fig = make_subplots(rows=2, cols=1, specs =[[{"secondary_y": True}],[{"secondary_y": True}]])
fig.add_trace(go.Scatter(name='PPO', x=df.index, y=df['PPO']), row=2,col=1)
fig.add_trace(go.Bar(name='PPO_hist', x=df.index, y=df['PPO_hist']), row=2, col=1, secondary_y=True)
fig.add_trace(go.Scatter(name='PPO_line', x=df.index, y=df['PPO_line']), row=2, col=1)
fig.add_trace(go.Candlestick(name='BRML3', x=df.index, close=df['Close'], open=df['Open'], high=df['High'], low=df['Low']))
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

###3.3. Rate of Change
####verificar se o mercado esta sobre vendido 
### ROC= [Fechamento - Fechamento de n periodos anteriores) / Fechamento de n periodos anteriores)] * 100

In [12]:
from ta.momentum import ROCIndicator

In [15]:
roc = ROCIndicator(df['Close'])
df['ROC'] = roc.roc()
#default da função são de 12 periodos ou seja 12 candlestick

In [16]:
fig = go.Figure()
    
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(name='ROC',x=df.index, y=df['ROC']), row=2, col=1)

fig.add_trace(go.Candlestick(name='BRML3',x=df.index, close=df['Close'], open=df['Open'],high=df['High'], low=df['Low']),row=1,col=1)
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

##4. Indicadores de Volume

###4.1. OBV - On Balance Volume

In [ ]:
from ta.volume import OnBalanceVolumeIndicator

In [ ]:
obv = OnBalanceVolumeIndicator(df['Close'], df['Volume'])
df['OBV'] = obv.on_balance_volume()

In [ ]:
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Bar(name='OBV', x=df.index, y=df['OBV']), row=2, col=1)
fig.add_trace(go.Candlestick(name='BRML3', x=df.index, close=df['Close'], open=df['Open'], high=df['High'], low=df['Low']), row=1, col=1)
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

###4.2. VWAP - Volume Weighted Average Price

In [ ]:
from ta.volume import VolumeWeightedAveragePrice

In [ ]:
vwap = VolumeWeightedAveragePrice(df['High'], df['Low'], df['Close'], df['Volume'])
df['VWAP'] = vwap.volume_weighted_average_price()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(name='VWAP', x=df.index, y=df['VWAP']))
fig.add_trace(go.Candlestick(name='BRML3', x=df.index, close=df['Close'], open=df['Open'], high=df['High'], low=df['Low']))
fig.update_layout(xaxis_rangeslider_visible=False, title_text='VWAP')
fig.show()

###4.3. Money Flow Index

In [ ]:
from ta.volume import MFIIndicator

In [ ]:
mfi = MFIIndicator(df['High'], df['Low'], df['Close'], df['Volume'])
df['MFI'] = mfi.money_flow_index()

In [ ]:
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(name='MFI', x= df.index, y=df['MFI']), row=2, col=1)
fig.add_trace(go.Candlestick(name='BRML3', x=df.index, close=df['Close'], open=df['Open'], high = df['High'], low=df['Low']), row=1, col=1)
fig.update_layout(xaxis_rangeslider_visible=False, title_text='MFI')
fig.show()

##5. Indicadores de Volatilidade

###5.1. Bandas de Bollinger

In [ ]:
from ta.volatility import BollingerBands

In [ ]:
bb = BollingerBands(close=df['Close'], window=20, window_dev=2)
df['bb_bbm'] = bb.bollinger_mavg()
df['bb_bbh'] = bb.bollinger_hband()
df['bb_bbl'] = bb.bollinger_lband()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(name='MédiaMovel', x= df.index, y=df['bb_bbm']))
fig.add_trace(go.Candlestick(name='BRML3', x=df.index, close=df['Close'], open=df['Open'], 
                             high = df['High'], low=df['Low']))
fig.add_trace(go.Scatter(name='Banda Superior', x=df.index, y=df['bb_bbh']))
fig.add_trace(go.Scatter(name='Banda inferior', x=df.index, y=df['bb_bbl']))
fig.update_layout(xaxis_rangeslider_visible=False, title_text='Bollinger Bands')
fig.show()

##5.2. ATR - Average True Range

In [ ]:
from ta.volatility import AverageTrueRange

In [ ]:
atr = AverageTrueRange(df['High'], df['Low'], df['Close'])

df['ATR'] = atr.average_true_range()

In [ ]:
fig = make_subplots(rows=2,cols=1)
fig.add_trace(go.Scatter(name='ATR', x=df.index, y=df['ATR']), row=2, col=1)
fig.add_trace(go.Candlestick(name='BRML3', x=df.index, close=df['Close'], open=df['Open'], high = df['High'], low=df['Low']), row=1, col=1)
fig.update_layout(xaxis_rangeslider_visible=False, title_text = 'ATR')
fig.show()

###5.3. Keltner Channel


In [ ]:
from ta.volatility import KeltnerChannel

In [ ]:
keltner = KeltnerChannel(df['High'], df['Low'], df['Close'])
df['KELTNER_HB'] = keltner.keltner_channel_hband()
df['KELTNER_LB'] = keltner.keltner_channel_lband()
df['KELTNER_M'] = keltner.keltner_channel_mband()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(name='Keltner MM', x=df.index, y=df['KELTNER_M']))
fig.add_trace(go.Candlestick(name='BRML3', x=df.index, close=df['Close'], open=df['Open'], high = df['High'], low=df['Low']))
fig.add_trace(go.Scatter(name='Banda Superior', x=df.index, y=df['KELTNER_HB']))
fig.add_trace(go.Scatter(name='Banda Inferior', x=df.index, y=df['KELTNER_LB']))
fig.update_layout(xaxis_rangeslider_visible=False, title_text='Keltner Channel')
fig.show()

#6. Indicadores de tendência


###6.1. MACD

In [ ]:
from ta.trend import MACD

In [ ]:
macd = MACD(df['Close'])
df['MACD_LINE'] = macd.macd()
df['MACD_DIFF'] = macd.macd_diff()


In [ ]:
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Bar(name='MACD', x=df.index, y=df['MACD_LINE']), row=2,col=1)
fig.add_trace(go.Scatter(name='MACD_DIFF', x=df.index, y=df['MACD_DIFF']), row=2, col=1)
fig.add_trace(go.Scatter(name='MACD_line', x=df.index, y=df['MACD_LINE']), row=2, col=1)
fig.add_trace(go.Candlestick(name='BRML3', x=df.index, close=df['Close'], open=df['Open'], high = df['High'], low=df['Low']),row=1,col=1)
fig.update_layout(xaxis_rangeslider_visible=False, title_text='MACD')
fig.show()

##5.2. Simple Moving Average

In [ ]:
from ta.trend import SMAIndicator

In [ ]:
sma_short = SMAIndicator(df['Close'], window=5)
sma_long = SMAIndicator(df['Close'], window=30)
df['SMA_SHORT'] = sma_short.sma_indicator()
df['SMA_LONG'] = sma_long.sma_indicator()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(name='SMA_SHORT', x=df.index, y=df['SMA_SHORT']))
fig.add_trace(go.Candlestick(name='BRML3', x=df.index, close=df['Close'], open=df['Open'], high = df['High'], low=df['Low']))
fig.add_trace(go.Scatter(name='SMA_LONG', x=df.index, y=df['SMA_LONG']))
fig.update_layout(xaxis_rangeslider_visible=False, title_text='Simple Moving Average')
fig.show()

##6.3. Média Móvel Exponencial - EMA

In [ ]:
from ta.trend import EMAIndicator

In [ ]:
ema_short = EMAIndicator(df['Close'], window=5)
ema_long = EMAIndicator(df['Close'], window=30)
df['EMA_SHORT'] = ema_short.ema_indicator()
df['EMA_LONG'] = ema_long.ema_indicator()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(name='EMA_SHORT', x=df.index, y=df['EMA_SHORT']))
fig.add_trace(go.Candlestick(name='BRML3', x=df.index, close=df['Close'], open=df['Open'], high = df['High'], low=df['Low']))
fig.add_trace(go.Scatter(name='EMA_LONG', x=df.index, y=df['EMA_LONG']))
fig.update_layout(xaxis_rangeslider_visible=False, title_text='EMA')
fig.show()